### 可変長引数を取れるようにFunctionクラスを修正

In [1]:
import numpy as np

In [2]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        return outputs
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [3]:
class Variable:
    def __init__(self, data):
        # 例外処理を入れる　ndarray以外のものが入ったらTypeError
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        
        while funcs:
            f = funcs.pop()  
            x, y = f.input, f.output  
            x.grad = f.backward(y.grad)  
            
            if x.creator is not None:
                funcs.append(x.creator)  # add the creator of the previous function to  the list


### Addクラスの実装

In [4]:
class Add(Function):
    def forward(self, xs):
        x0, x1 = xs
        y = x0 + x1
        return (y,)

In [5]:
xs = [Variable(np.array(2.0)), Variable(np.array(3.0))]
f = Add()
ys = f(xs)  # ysはタプル
y = ys[0]  # ysはVariableインスタンスのリスト
print(y.data)

5.0


### 改善点①

In [6]:
class Function:
    def __call__(self, *inputs):  # アスタリスク
        xs = [x.data for x in inputs]
        ys = self.forward(xs)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        
        # リストの要素が一つのときは最初の要素を返す
        return outputs if len(outputs) > 1 else outputs[0]
        # return outputs
    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [7]:
def f(*x):  # アスタリスクをつけることで任意の数の引数を取ることができる  tupleで渡されるよう
    print(x)
    print(type(x))
    
f(1, 2, 3)
f(1, 2, 4,7777)

(1, 2, 3)
<class 'tuple'>
(1, 2, 4, 7777)
<class 'tuple'>


In [8]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)   # 帰ってくるのはVariableインスタンスなので値は取り出さないといけない

TypeError: __call__() takes 2 positional arguments but 3 were given

### 改善点②

In [9]:
class Function:
    def __call__(self, *inputs):   # *inputsは可変長（何個でもいい）のVariableインスタンス
        xs = [x.data for x in inputs]
        print(xs)
        ys = self.forward(*xs)  # アスタリスクをつけてアンパッキング
        if not isinstance(ys, tuple):# tupleではない場合の追加対応  tupleにしておくことでlenが取れる
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
                
        return outputs if len(outputs) > 1 else outputs[0]

    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [10]:
class Add(Function):
    def forward(self, x0, x1):
        # x0, x1 = xs
        y = x0 + x1
        # yのまま返していい
        return y

In [11]:
def add(x0, x1):
    return Add()(x0, x1)

In [12]:
x0=Variable(np.array(2))
x1=Variable(np.array(3))

y = add(x0, x1)
print(y.data)  # 最終的にはVariableインスタンスで帰ってくる

[array(2), array(3)]
5


### 掛け算も実装してみる

In [13]:
class Add(Function):
    def forward(self, x0, x1):
        # x0, x1 = xs
        y = x0 + x1
        # yのまま返していい
        return y

In [14]:
#  classをdef にしていて 下のセルを実行するときにMultipl() missing 1 required positional argument: 'Function'のエラーが消えなかった
# defだと継承ではなく引数としてFunctionが見られるのでargument missingしていた

class Multipl(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
def multipl(x0, x1):
    return Multipl()(x0,x1)

In [15]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
y = multipl(x0, x1)
print(y.data)

[array(2), array(3)]
6


### 可変長の引数逆伝播編

In [16]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    # 足し算ではそのまま返す
    def backward(self, gy):
        return gy, gy

In [17]:
a = (1, 2, 3)
f = lambda x: x**2
f(a)

TypeError: unsupported operand type(s) for ** or pow(): 'tuple' and 'int'

In [18]:
class Variable:
    def __init__(self, data):
        # 例外処理を入れる　ndarray以外のものが入ったらTypeError
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        print(len(funcs))
        
        while funcs:
            f = funcs.pop()  
            gys = [output.grad for output in f.outputs]
            print('type(gys)', type(gys))
            gxs = f.backward(*gys)
            print('type(gxs)', type(gxs))
            if not isinstance(gxs, tuple):
                gxs = (gxs, )
            print(f.inputs)   
            for x, gx in zip(f.inputs, gxs):
                print('x', x.data)
                print('type(x)',type(x))
                print('type(gx)',type(gx))
                x.grad = gx
            
                if x.creator is not None:
                    funcs.append(x.creator)  # add the creator of the previous function to  the list


In [19]:
# Suquare暮らすの実装

class Square(Function):
    def forward(self, x):
        y = x**2
        return y
        
    def backward(self, gy):
        x = self.inputs[0].data  
        gx = 2*x*gy
        return gx
    

In [20]:
def square(x):
    f = Square()
    return f(x)
# return Square()(x)  でも可

def exp(x):
    f = Exp()
    return f(x)
# return Exp()(x)  でも可
    

In [21]:
x = Variable(np.array(2.0))
y = Variable(np.array(3.0))

z = add(square(x), square(y))
z.backward()
print(z.data)
print(x.grad)
print(y.grad)

[array(2.)]
[array(3.)]
[array(4.), array(9.)]
1
type(gys) <class 'list'>
type(gxs) <class 'tuple'>
(<__main__.Variable object at 0x10a8f2a10>, <__main__.Variable object at 0x10a8f2f50>)
x 4.0
type(x) <class '__main__.Variable'>
type(gx) <class 'numpy.ndarray'>
x 9.0
type(x) <class '__main__.Variable'>
type(gx) <class 'numpy.ndarray'>
type(gys) <class 'list'>
type(gxs) <class 'numpy.float64'>
(<__main__.Variable object at 0x10a981bd0>,)
x 3.0
type(x) <class '__main__.Variable'>
type(gx) <class 'numpy.float64'>
type(gys) <class 'list'>
type(gxs) <class 'numpy.float64'>
(<__main__.Variable object at 0x10a981ed0>,)
x 2.0
type(x) <class '__main__.Variable'>
type(gx) <class 'numpy.float64'>
13.0
4.0
6.0


### 同じ変数を繰り返し使う

In [22]:
class Variable:
    def __init__(self, data):
        # 例外処理を入れる　ndarray以外のものが入ったらTypeError
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        
        while funcs:
            f = funcs.pop()  
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )  
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad+gx
            
                if x.creator is not None:
                    funcs.append(x.creator) 


In [23]:
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()

print(x.grad)

[array(3.), array(3.)]
2.0


In [24]:
x = Variable(np.array(3.0))
y = add(add(x,x), x)
y.backward()

print(x.grad)

[array(3.), array(3.)]
[array(6.), array(3.)]
3.0


### 微分のリセット

In [25]:
# ①回目の計算
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()

print(x.grad)


# ②回めの計算
y = add(add(x,x), x)
y.backward()

print(x.grad)

# 2回目の微分は①回目の微分に加算されている
# 防ぐために、gradを初期化するメソッドを追加

[array(3.), array(3.)]
2.0
[array(3.), array(3.)]
[array(6.), array(3.)]
5.0


In [26]:
class Variable:
    def __init__(self, data):
        # 例外処理を入れる　ndarray以外のものが入ったらTypeError
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
    
    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        
        while funcs:
            f = funcs.pop()  
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )  
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad+gx
            
                if x.creator is not None:
                    funcs.append(x.creator) 
                    
    def cleargrad(self):
        self.grad = None

In [27]:
# ①回目の計算
x = Variable(np.array(3.0))
y = add(x,x)
y.backward()

print(x.grad)


# ②回めの計算 初めに使い回す変数のgradを初期化する
x.cleargrad()
y = add(add(x,x), x)
y.backward()

print(x.grad)

[array(3.), array(3.)]
2.0
[array(3.), array(3.)]
[array(6.), array(3.)]
3.0


### 枝分かれしてつながるような複雑な計算グラフでbackwardができるようにするために世代を追加する

In [28]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))
        
        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0　　# 世代を追加
    
    def set_creator(self, func):
        self.creator = func
        self.genaration = func.generation +1  # 親の世代＋１を代入
        
    def backward(self):
        if self.grad == None:
            self.grad = np.ones_like(self.data)
        
        funcs = [self.creator]
        
        while funcs:
            f = funcs.pop()  
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs, )  
                
            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad+gx
            
                if x.creator is not None:
                    funcs.append(x.creator) 
                    
    def cleargrad(self):
        self.grad = None
    

SyntaxError: invalid character in identifier (<ipython-input-28-b3de3263e030>, line 10)

In [29]:
class Function:
    def __call__(self, *inputs):   
        xs = [x.data for x in inputs]
        print(xs)
        ys = self.forward(*xs)  
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        self.generation = max([x.generation for x in inputs])  # すべてのinputの世代をリストにして最大を求める
        
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
                
        return outputs if len(outputs) > 1 else outputs[0]

    
    def forward(self, xs):
        raise NotImplementedError()
        
    def backward(self, gys):
        raise NotImplementedError()

In [30]:
generations = [2,0,1,4,2]
funcs = []

for g in generations:
    f = Function()  # ダミーの関数クラス
    f.generation = g
    funcs.append(f)

[f.generation for f in funcs]

[2, 0, 1, 4, 2]

In [31]:
# funcsを並び替える
funcs.sort(key=lambda x: x.generation)
[f.generation for f in funcs]

[0, 1, 2, 2, 4]

In [32]:
# popで後ろ関数を取り出す
f = funcs.pop()
f.generation

4

In [33]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

In [34]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    # 足し算ではそのまま返す
    def backward(self, gy):
        return gy, gy
    
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
        
    def backward(self, gy):
        x = self.inputs[0].data  
        gx = 2*x*gy
        return gx
    
def square(x):
    f = Square()
    return f(x)
# return Square()(x)  でも可

def exp(x):
    f = Exp()
    return f(x)

def add(x0, x1):
    return Add()(x0, x1)

    
x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

[array(2.)]
[array(4.)]
[array(4.)]
[array(16.), array(16.)]
32.0
64.0


In [35]:
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


x = Variable(np.array(2.0))
a = square(x)
y = add(square(a), square(a))
y.backward()

print(y.data)
print(x.grad)

32.0
64.0


### メモリ管理と循環参照

In [37]:
import weakref

a = np.array([1,2,3])
b = weakref.ref(a)

b

<weakref at 0x10a98dc50; to 'numpy.ndarray' at 0x10a99fee0>

In [38]:
b()

array([1, 2, 3])

In [39]:
a = None
b
# python処理系ではなくjupyter notebookの処理系で動いており追加の参照を保持するのでdeadにはならない

<weakref at 0x10a98dc50; to 'numpy.ndarray' at 0x10a99fee0>

In [62]:
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        # outputs はlist　中のVariableインスタンスに対する参照をweakrefにする
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()



In [51]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            #output()で参照しているオブジェクトにたどり着く。outputのままだと参照しているオブジェクトのメモリに行く
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

循環参照解消　<br>
動作確認

In [45]:
for i in range(10):
    x = Variable(np.random.randn(10000))  # 巨大なデータ
    y = square(square(square(x)))  # 複雑な計算
        


# jupyter notebookからはできなかった

### メモリ使用量を減らすモードの設定
- 不要な微分（中間変数の微分）をすぐに消す
- 逆伝播が必要ない場合のモード

In [72]:
class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            #output()で参照しているオブジェクトにたどり着く。outputのままだと参照しているオブジェクトのメモリに行く
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
                    
            if not retain_grad:
                for y in f.outputs:
                    # 末端のgradがあればいいのでもとのgradはreset
                    y().grad = None  # yはweakref  y()でないとVariableオブジェクトにたどり着かない
                    # 微分のデータへの参照は０
                    
                    

In [67]:
x0 = Variable(np.array(1.0))
x1 = Variable(np.array(1.0))

t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)  # ここは保持されていないはず
print(x0.grad, x1.grad)

None None
2.0 1.0


### configクラスによる切り替え

In [68]:
class Config:
    enable_backprop = True
    

In [69]:
import weakref

class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])  # set genaration
            for output in outputs:
                output.set_creator(self)  # set connection
            
        self.inputs = inputs
        # outputs はlist　中のVariableインスタンスに対する参照をweakrefにする
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()



In [70]:


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)

In [74]:
Config.enable_backprop = True
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()

Config.enable_backprop = False
x = Variable(np.ones((100,100,100)))
y = square(square(square(x)))
y.backward()

AttributeError: 'NoneType' object has no attribute 'generation'